This notebooks uses data from [ESA's MASTER tool](https://sdup.esoc.esa.int/) to create a population of small debris fragments between 1 and 10 cm.

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm 
from sklearn.neighbors import KernelDensity

starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
np.random.seed(42)
earth_radius = 6371
sats = pd.read_csv("../data/initial_population.csv")
N_base = len(sats)
# Utility functions
def get_shell_volume(lower_altitude,upper_altitude):
    # Subtraction of sphere volumes to get shell volume
    return 4/3 * np.pi * ((earth_radius+upper_altitude)**3-(earth_radius+lower_altitude)**3)

def get_num_of_parts_in_shell(lower_altitude,upper_altitude,density):
    # Shell volume * density
    volume = get_shell_volume(lower_altitude,upper_altitude)
    nr = volume * density
#     print(volume,density,nr)
    return nr

# Get estimated number of debris particles depending on altitude and size

In [ ]:
# Read in density data for 1-2.5cm, 2.5-5cm , ...
# Note it is expected that each file has densities for the same altitudes, this is not checked.
data_1cm = pd.read_csv("../data/MASTER_small_debris/1-2.5cm.csv")
data_2cm = pd.read_csv("../data/MASTER_small_debris/2.5-5cm.csv")
data_5cm = pd.read_csv("../data/MASTER_small_debris/5-7.5cm.csv")
data_7cm = pd.read_csv("../data/MASTER_small_debris/7.5-10cm.csv")

In [ ]:
data_1cm

In [ ]:
num_debris_per_size = {} # will hold counts per altitude & size
sizes = [[1,2.5],[2.5,5.],[5,7.5],[7.5,10.0]]
datasets = [data_1cm,data_2cm,data_5cm,data_7cm]
shell_lower_alt = data_1cm["Altitude"].values[:-1]
shell_upper_alt = data_1cm["Altitude"].values[1:]
total_num_of_particles = 0

In [ ]:
for size,data in zip(sizes,datasets):
    # Get bounds of each shell in which we will investigate density
    num_debris_per_altitude = []
    densities = data["Density"].values
    for lower,upper,density in zip(shell_lower_alt,shell_upper_alt,densities):
        num = get_num_of_parts_in_shell(lower,upper,density)
        total_num_of_particles += num
        num_debris_per_altitude.append(num)
    num_debris_per_size[size[0]] = num_debris_per_altitude
print("In total expecting ",total_num_of_particles, " debris particles.")

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=150)
fig.patch.set_facecolor('white')
names = []
for s,nums in num_debris_per_size.items():
    plt.bar(shell_lower_alt,nums,width = 20.0)
    names.append("> "+str(s) + "cm")
plt.legend(names)
plt.xlabel("Altitude [km]")
plt.ylabel("# of particles")

# Sample from above distribution (+ inclination & eccentricity from base population)

## Collect inclination & eccentricity of base pop

In [ ]:
incl_base_pop = []
ecc_base_pop = []
for _,s in sats.iterrows():
    r = np.array([s["r_x[km]"],s["r_y[km]"],s["r_z[km]"]]) * 1000.
    v =  np.array([s["v_x[km]"],s["v_y[km]"],s["v_z[km]"]]) * 1000.
    _,e,i,_,_,_ = pk.ic2par(r,v,mu=pk.MU_EARTH)
    incl_base_pop.append(i)
    ecc_base_pop.append(e)

## Use kernel density estimators to sample from base pop distributions of i and e

In [ ]:
kde_incl = KernelDensity(kernel='gaussian',bandwidth=5e-4).fit(np.asarray(incl_base_pop).reshape(-1, 1))
kde_ecc = KernelDensity(kernel='gaussian',bandwidth=5e-4).fit(np.asarray(ecc_base_pop).reshape(-1, 1))

In [ ]:
test_samples_incl = kde_incl.sample(len(sats))
test_samples_ecc = kde_ecc.sample(len(sats))
N_bins = 100
fig = plt.figure(figsize=(5,2),dpi=150)
fig.patch.set_facecolor('white')
plt.hist(incl_base_pop,bins=N_bins)
plt.hist(test_samples_incl,bins=N_bins)
plt.xlabel("Inclination[rad]")
plt.ylabel("Frequency")
plt.legend(["Original","KDE"])
fig = plt.figure(figsize=(5,2),dpi=150)
fig.patch.set_facecolor('white')
plt.hist(ecc_base_pop,bins=N_bins)
plt.hist(test_samples_ecc,bins=N_bins)
plt.xlabel("Eccentricity[rad]")
plt.ylabel("Frequency")
plt.legend(["Original","KDE"])

In [ ]:
def sample_incl():
    return np.maximum(0,kde_incl.sample(1)[0][0])

def sample_ecc():
    return np.maximum(0,kde_ecc.sample(1)[0][0])

## Create our debris population

In [ ]:
population = []
debris_index = N_base
for (_,nums),size in zip(num_debris_per_size.items(),sizes):
    for altitude,alt_upper_bound,N_at_altitude in tqdm(zip(shell_lower_alt,shell_upper_alt,nums),total=len(shell_lower_alt)):
        
        N_at_altitude = int(np.round(N_at_altitude))
        if N_at_altitude == 0:
            continue
        # Sampling half size for radius and converting to m
        sampled_radii = np.random.uniform(low=size[0] / 2.0 / 100.0, high=size[1] / 2.0 / 100.0,size=(N_at_altitude))
        
        # Sample individual satellites
        for idx in range(N_at_altitude):
            n = str(debris_index)
            sat = {"COSPAR_ID": n,"NAME": n, "TYPE": "passive"}
            sat["RADIUS[m]"] = sampled_radii[idx]
            if sat["RADIUS[m]"] > 0.01:
                sat["M[kg]"] = 4 / 3 * np.pi *(sat["RADIUS[m]"])**3 * 92.937 * (2*sat["RADIUS[m]"])**(-0.74)
            else:
                sat["M[kg]"] = 4 / 3 * np.pi *(sat["RADIUS[m]"])**3 * 2698.9
                
            # Approximate BSTAR as p0 * C_D * 2*pi*r / (2 * mass * earth_radius[m])
            sat["BSTAR[1 / Earth Radius]"] = (0.1570 * 2.2 * np.pi * sat["RADIUS[m]"]**2) / (2*sat["M[kg]"]*earth_radius)
            
            # Sample orbital elements
            sampled_sma = earth_radius*1000.0 + np.random.uniform(low=altitude*1000,high=alt_upper_bound*1000)
            w = np.random.uniform(0, 2*np.pi)
            W = np.random.uniform(0, 2*np.pi)
            E = np.random.uniform(0, 2*np.pi)
            el = [sampled_sma,sample_ecc(),sample_incl(),W,w,E]
            planet = pk.planet.keplerian(starting_t, el, pk.MU_EARTH, 1.0, 1.0, 1.0,f"deb_{debris_index}")
            pos,v = planet.eph(starting_t)
            sat["r_x[km]"] = pos[0] / 1000.0
            sat["r_y[km]"] = pos[1] / 1000.0
            sat["r_z[km]"] = pos[2] / 1000.0
            sat["v_x[km]"] = v[0] / 1000.0
            sat["v_y[km]"] = v[1] / 1000.0
            sat["v_z[km]"] = v[2] / 1000.0
            population.append(sat)
            debris_index += 1

In [ ]:
debris_v = np.array([np.sqrt(p["v_x[km]"]**2+p["v_y[km]"]**2+p["v_z[km]"]**2) for p in population])
sat_v = np.array([np.sqrt(p["v_x[km]"]**2+p["v_y[km]"]**2+p["v_z[km]"]**2) for _,p in sats.iterrows()])

## Plot some data on it

In [ ]:
fig = plt.figure(figsize=(5,4),dpi=150)
fig.patch.set_facecolor('white')
plt.hist(debris_v,bins=N_bins,density=True,alpha=0.5)
plt.hist(sat_v,bins=N_bins,density=True,alpha=0.5)
plt.xlabel("|v|[km]")
plt.ylabel("Relative Density")
plt.legend(["Debris","Base Pop"])

In [ ]:
debris_r = np.array([p["RADIUS[m]"] for p in population])
sat_r = np.array([p["RADIUS[m]"]  for _,p in sats.iterrows()])
fig = plt.figure(figsize=(5,4),dpi=150)
fig.patch.set_facecolor('white')
plt.hist(debris_r,bins=N_bins,density=True,alpha=0.5)
plt.hist(sat_r,bins=N_bins,density=True,alpha=0.5)
plt.xscale("log")
plt.xlabel("Radius[m]")
plt.ylabel("Relative Density")
plt.legend(["Debris","Base Pop"])

In [ ]:
debris_positions = np.array([[p["r_x[km]"],p["r_y[km]"],p["r_z[km]"]] for p in population])
sat_positions = np.array([[p["r_x[km]"],p["r_y[km]"],p["r_z[km]"]] for _,p in sats.iterrows()])


fig = plt.figure(figsize=(10,10),dpi=150)
fig.patch.set_facecolor('white')
ax = plt.axes(projection='3d');
ax.set_xlabel("X[km]")
ax.set_ylabel("Y[km]")
ax.set_zlabel("Z[km]")

u, v = np.mgrid[0:2*np.pi:40j, 0:np.pi:40j]
x = np.cos(u)*np.sin(v) * 6371
y = np.sin(u)*np.sin(v) * 6371
z = np.cos(v) * 6371
ax.plot_surface(x, y, z, color="blue",alpha=0.5)
ax.scatter(debris_positions[:,0],debris_positions[:,1],debris_positions[:,2],".",s=1,alpha=0.025,color="black")
ax.scatter(sat_positions[:,0],sat_positions[:,1],sat_positions[:,2],".",s=1,alpha=0.5,color="red")

In [ ]:
df = pd.DataFrame(population)

In [ ]:
fig = plt.figure(figsize=(6,3),dpi=150)
fig.patch.set_facecolor('white')
p = sns.histplot(data=df,x="BSTAR[1 / Earth Radius]")

## Load base population and append to it

In [ ]:
base_df = pd.read_csv("../data/initial_population.csv")
base_df = base_df.drop(columns="Unnamed: 0")

In [ ]:
fig = plt.figure(figsize=(6,3),dpi=150)
fig.patch.set_facecolor('white')
p = sns.histplot(data=base_df,x="BSTAR[1 / Earth Radius]")
p.set_xscale("log")

In [ ]:
new_pop = pd.concat([base_df,df])

In [ ]:
new_pop = new_pop.reset_index()
new_pop = new_pop.drop(columns="index")
new_pop

In [ ]:
new_pop.to_csv("../data/initial_population_and_1cm_debris.csv")